# Exploring data of saraevLCAServersR65152021

In [1]:
import pandas as pd
from itables import show

df = pd.read_csv("table3-13-16.csv", header=[0,1])

In [2]:
# Flatten column name and column unit
df.columns = df.columns.map("{0[0]}({0[1]})".format)
df

,Device(ID),Workload(name),Load(mode),Duration(hours),Power draw(W),Lifespan(yr),Energy(kWh/yr)
0,R6515,Light-medium,100%,2.4,244.2,4,213.92
1,R6515,Light-medium,50%,8.4,194.1,4,595.11
2,R6515,Light-medium,10%,7.2,132.1,4,347.16
3,R6515,Light-medium,Idle,6.0,94.7,4,207.39
4,R6515,Heavy,100%,3.6,244.2,4,320.88
5,R6515,Heavy,50%,13.2,194.1,4,935.17
6,R6515,Heavy,10%,4.8,132.1,4,231.44
7,R6515,Heavy,Idle,2.4,94.7,4,82.96
8,R7515,Light-medium,100%,2.4,281.5,4,246.59
9,R7515,Light-medium,50%,8.4,228.4,4,700.27


## Energy per device and load-mode

In [3]:
import plotly.express as px

fig = px.bar(df, x="Device(ID)", y="Energy(kWh/yr)", color="Load(mode)", text_auto=True, facet_col="Workload(name)")
#fig.update_traces(textinfo="label+percent root")
fig.show()

## Time spent in load-mode and power draw per device

In [4]:
import plotly.express as px

fig = px.bar(df, x="Device(ID)", color="Power draw(W)", y="Duration(hours)", text="Load(mode)", facet_col="Workload(name)")
fig.show()

## Time spent in load-mode and energy consumption per device

In [5]:
import plotly.express as px

fig = px.bar(df, x="Device(ID)", color="Energy(kWh/yr)", y="Duration(hours)", text="Load(mode)", facet_col="Workload(name)")
fig.show()

In [17]:
import plotly.express as px

fig = px.bar(df, x="Device(ID)", color="Power draw(W)", y="Energy(kWh/yr)", text="Load(mode)", facet_col="Workload(name)")
fig.show()

Performance specs @ https://downloads.dell.com/manuals/common/poweredge_perf_amdepyc7002series.pdf

```{list-table}
* - Model
  - GFLOPs
  - Stream Triad (MB/s)
  - SPECint_rate_base2017
  - SPECfp_rate_base2017
* - R6515
  - 1244
  - 164316
  - 204
  - 177
* - R6525
  - 2385
  - 353000 (best)
  - NA
  - NA
```

## Data from the SPEC benchmarks for these servers

In [10]:
spec = pd.read_csv("specint2017.csv")
spec

,Device(ID),URL,base,peak,FPbase,FPpeak
0,R6515,https://www.spec.org/cpu2017/results/res2019q3...,211,229,177,195
1,R6525,https://www.spec.org/cpu2017/results/res2019q4...,377,405,354,391
2,R7515,https://www.spec.org/cpu2017/results/res2019q4...,206,223,173,189
3,R7525,https://www.spec.org/cpu2017/results/res2019q4...,418,452,361,392


Merging this data into the Dell LCA data gives (and using an idle load of 1%):

In [11]:
spec = spec.merge(df,on="Device(ID)")
load = pd.DataFrame({"Load(mode)":["Idle", "10%", "50%", "100%"], "load": [0.01,0.1,0.5,1]})
spec = spec.merge(load, on="Load(mode)")
spec["PPE"] = spec["load"]*spec["base"]/spec["Energy(kWh/yr)"]
spec["PPEfp"] = spec["load"]*spec["FPbase"]/spec["Energy(kWh/yr)"]
spec

,Device(ID),URL,base,peak,FPbase,FPpeak,Workload(name),Load(mode),Duration(hours),Power draw(W),Lifespan(yr),Energy(kWh/yr),load,PPE,PPEfp
0,R6515,https://www.spec.org/cpu2017/results/res2019q3...,211,229,177,195,Light-medium,100%,2.4,244.2,4,213.92,1.00,0.986350,0.827412
1,R6515,https://www.spec.org/cpu2017/results/res2019q3...,211,229,177,195,Heavy,100%,3.6,244.2,4,320.88,1.00,0.657567,0.551608
2,R6525,https://www.spec.org/cpu2017/results/res2019q4...,377,405,354,391,Light-medium,100%,2.4,449.8,4,394.02,1.00,0.956804,0.898432
3,R6525,https://www.spec.org/cpu2017/results/res2019q4...,377,405,354,391,Heavy,100%,3.6,449.8,4,591.04,1.00,0.637859,0.598944
4,R7515,https://www.spec.org/cpu2017/results/res2019q4...,206,223,173,189,Light-medium,100%,2.4,281.5,4,246.59,1.00,0.835395,0.701569
5,R7515,https://www.spec.org/cpu2017/results/res2019q4...,206,223,173,189,Heavy,100%,3.6,281.5,4,369.89,1.00,0.556922,0.467707
6,R7525,https://www.spec.org/cpu2017/results/res2019q4...,418,452,361,392,Light-medium,100%,2.4,389.0,4,340.76,1.00,1.226670,1.059397
7,R7525,https://www.spec.org/cpu2017/results/res2019q4...,418,452,361,392,Heavy,100%,3.6,389.0,4,511.15,1.00,0.817764,0.706251
8,R6515,https://www.spec.org/cpu2017/results/res2019q3...,211,229,177,195,Light-medium,50%,8.4,194.1,4,595.11,0.50,0.177278,0.148712
9,R6515,https://www.spec.org/cpu2017/results/res2019q3...,211,229,177,195,Heavy,50%,13.2,194.1,4,935.17,0.50,0.112814,0.094635


## Workload distribution per device

In [7]:
import plotly.express as px

fig = px.bar(spec, x="Device(ID)", y="PPE", color="Load(mode)", text_auto=True, facet_col="Workload(name)")
#fig.update_traces(textinfo="label+percent root")
fig.show()

## Workload distribution per device on floating-point

In [8]:
import plotly.express as px

fig = px.bar(spec, x="Device(ID)", y="PPEfp", color="Load(mode)", text_auto=True, facet_col="Workload(name)")
#fig.update_traces(textinfo="label+percent root")
fig.show()

The following table is for the same model, but with AMD EPYC 7702 2.00GHz

In [12]:
specssj = pd.read_csv("specssj2008.csv")
specssj


,Device(ID),Ops,Idle Watts,100% Watts
0,R6515,6.124.305,54.5,231
1,R6525,11.115.782,81.6,404
2,R7515,5.676.534,99.5,246
3,R7525,11.118.115,72.3,393
